# Imports

In [4]:
# for working on google colab:
#!wget -q "https://github.com/sh0w/machine-learning-pattern-classfication/raw/main/task3_classification/dataset/dataset_with_predicted_mean_A_mean_V.csv" -O "dataset.csv"
#df = pd.read_csv("dataset.csv")

In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [6]:

# does not contain predicted mean_A and mean_V values:
# df = pd.read_csv("dataset/phase_3_TRAIN_7d499bff69ca69b6_6372c3e_MLPC2021_generic.csv")

# contains our predicted meanA and meanV:
df = pd.read_csv("dataset/dataset_with_predicted_mean_A_mean_V.csv")


In [7]:
df.head()

,id,essentia_dissonance_mean,essentia_dissonance_stdev,essentia_dynamic_complexity,essentia_loudness,essentia_onset_rate,essentia_pitch_salience_mean,essentia_pitch_salience_stdev,essentia_spectral_centroid_mean,essentia_spectral_centroid_stdev,essentia_spectral_complexity_mean,essentia_spectral_complexity_stdev,essentia_spectral_rolloff_mean,essentia_spectral_rolloff_stdev,essentia_strong_peak_mean,essentia_strong_peak_stdev,librosa_bpm,librosa_chroma_mean_0,librosa_chroma_var_0,librosa_chroma_pct_10_0,librosa_chroma_pct_50_0,librosa_chroma_pct_90_0,librosa_chroma_mean_1,librosa_chroma_var_1,librosa_chroma_pct_10_1,librosa_chroma_pct_50_1,librosa_chroma_pct_90_1,librosa_chroma_mean_2,librosa_chroma_var_2,librosa_chroma_pct_10_2,librosa_chroma_pct_50_2,librosa_chroma_pct_90_2,librosa_chroma_mean_3,librosa_chroma_var_3,librosa_chroma_pct_10_3,librosa_chroma_pct_50_3,librosa_chroma_pct_90_3,librosa_chroma_mean_4,librosa_chroma_var_4,librosa_chroma_pct_10_4,...,librosa_chroma_pct_90_7,librosa_chroma_mean_8,librosa_chroma_var_8,librosa_chroma_pct_10_8,librosa_chroma_pct_50_8,librosa_chroma_pct_90_8,librosa_chroma_mean_9,librosa_chroma_var_9,librosa_chroma_pct_10_9,librosa_chroma_pct_50_9,librosa_chroma_pct_90_9,librosa_chroma_mean_10,librosa_chroma_var_10,librosa_chroma_pct_10,librosa_chroma_pct_50_10,librosa_chroma_pct_90_10,librosa_chroma_mean_11,librosa_chroma_var_11,librosa_chroma_pct_10_11,librosa_chroma_pct_50_11,librosa_chroma_pct_90_11,librosa_spectral_bandwidth_mean,librosa_spectral_bandwidth_stdev,librosa_spectral_flatness_mean,librosa_spectral_flatness_stdev,midlevel_features_melody,midlevel_features_articulation,midlevel_features_rhythm_complexity,midlevel_features_rhythm_stability,midlevel_features_dissonance,midlevel_features_tonal_stability,midlevel_features_minorness,score_mode,score_key_strength,mean_A,mean_V,quadrant,predicted_mean_A,predicted_mean_V,predicted_quadrant
0,GG-01-000,0.206445,0.131383,13.998791,58.329521,3.1,0.539736,0.193987,3367.790527,3684.124268,2.986095,2.530877,4423.862305,6692.707031,0.316206,0.329171,246.09375,0.423029,0.146997,0.008853,0.287094,1.000000,0.339058,0.089543,0.007493,0.289652,0.783410,0.372997,0.151724,0.008017,0.119889,1.0,0.380705,0.097001,0.015079,0.304984,0.855147,0.471226,0.157847,0.020450,...,1.0,0.295291,0.087601,0.004258,0.190495,0.764857,0.300751,0.138661,0.001662,0.058367,0.975641,0.253962,0.094545,0.004239,0.074733,0.765817,0.301682,0.099494,0.006088,0.144401,0.815172,3455.577210,1781.705789,0.139911,0.248388,0.065397,-0.029380,-0.303987,-0.053306,-0.325228,0.347745,-0.119278,0.0,0.795727,50.689655,2.724138,1,45.216172,1.749838,4
1,GG-01-001,0.145753,0.057971,3.533264,75.166183,4.2,0.476832,0.123564,1395.528809,401.987610,4.181923,2.296265,691.307373,194.689529,0.438665,0.363517,246.09375,0.270370,0.112473,0.008509,0.087995,0.937823,0.191162,0.039303,0.008384,0.107667,0.472149,0.280426,0.145651,0.011143,0.075144,1.0,0.281440,0.038196,0.023332,0.270883,0.559445,0.433210,0.177601,0.013791,...,1.0,0.141696,0.026381,0.005053,0.049705,0.425738,0.124960,0.059223,0.004241,0.040996,0.311990,0.128249,0.030747,0.005173,0.045485,0.455922,0.203408,0.069066,0.006298,0.093133,0.499201,2745.545697,721.292781,0.000275,0.000484,0.353040,-0.073957,-0.381182,0.120750,-0.479266,0.451572,-0.064252,0.0,0.844387,50.689655,2.724138,1,42.078254,0.360194,4
2,GG-01-002,0.149111,0.059011,3.337368,82.753929,4.3,0.457741,0.132359,1318.759644,289.306152,4.303592,2.127036,674.839294,210.393951,0.534696,0.410875,246.09375,0.193968,0.078629,0.003699,0.045964,0.661004,0.196718,0.045585,0.006544,0.134951,0.538369,0.394663,0.185849,0.008034,0.111176,1.0,0.252466,0.056374,0.015938,0.169116,0.649735,0.160429,0.087263,0.009318,...,1.0,0.199484,0.028600,0.010491,0.160100,0.427959,0.275843,0.136039,0.009648,0.091636,1.000000,0.213787,0.054531,0.008481,0.097247,0.541696,0.172331,0.083168,0.005382,0.043987,0.575106,2593.570349,583.569947,0.000202,0.000314,0.201885,0.016944,-0.337422,0.103853,-0.388214,0.430684

# Split Dataset to X and y=score_mode

In [11]:
# drop target value, student annotations and string ID from input features:
# all that is left are the music features:
X = df.drop(columns=['quadrant','mean_A','mean_V','id','score_mode','score_key_strength'])

# we want to predict the major/minor label:
y = df["score_mode"]

# Perform GridSearchCV to find best parameters for a Neural Network (MLPClassifier)

In [29]:

# Define a pipeline to search for the best combination of PCA truncation
# and classifier regularization.
from sklearn.decomposition import PCA, KernelPCA

mlp_gs = MLPClassifier(max_iter=230)

parameter_space = [{
    'nn__hidden_layer_sizes': [(5,10), (10,20,10), (5,7,5), (5,7), (10,5) ],
    'nn__activation': ['tanh', 'relu', 'sigmoid'],
    'nn__solver': ['sgd', 'adam', 'lbfgs'],
    'nn__alpha': [0.0005, 0.005, 0.05],
    'nn__learning_rate': ['adaptive','constant']
}]

pipeline = Pipeline([
        ("std_scaler", StandardScaler()),
        ('nn', mlp_gs)
      ])

In [30]:
clf_v = RandomizedSearchCV(pipeline, parameter_space, n_jobs=-1, cv=5, return_train_score=True, verbose=4, n_iter=100)

clf_v.fit(X, y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   28.9s
[Parallel(n_jobs=-1)]: Done 110 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 404 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  5.9min finished
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('std_scaler',
                                              StandardScaler(copy=True,
                                                             with_mean=True,
                                                             with_std=True)),
                                             ('nn',
                                              MLPClassifier(activation='relu',
                                                            alpha=0.0001,
                                                            batch_size='auto',
                                                            beta_1=0.9,
                                                            beta_2=0.999,
                                                            early_stopping=False,
                                                            epsilon=1e-08,
                                 

In [31]:
print("\n"+"#"*50,"\nbest estimator: ", clf_v.best_estimator_,"\n"+"#"*50,"\nbest params: ", clf_v.best_params_, "\n"+"#"*50,"\nbest score: ",clf_v.best_score_, "\n")



################################################## 
best estimator:  Pipeline(memory=None,
         steps=[('std_scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('nn',
                 MLPClassifier(activation='relu', alpha=0.05, batch_size='auto',
                               beta_1=0.9, beta_2=0.999, early_stopping=False,
                               epsilon=1e-08, hidden_layer_sizes=(10, 20, 10),
                               learning_rate='constant',
                               learning_rate_init=0.001, max_fun=15000,
                               max_iter=230, momentum=0.9, n_iter_no_change=10,
                               nesterovs_momentum=True, power_t=0.5,
                               random_state=None, shuffle=True, solver='lbfgs',
                               tol=0.0001, validation_fraction=0.1,
                               verbose=False, warm_start=False))],
         verbose=False) 
#####################

In [32]:
score_mode_predicted = clf_v.predict(X)


In [33]:
abs_error = [abs(y[i] - score_mode_predicted[i]) for i in range(2607)]
mean_error = np.mean(abs_error)

print("MEAN ERROR:  \t", mean_error)

MEAN ERROR:  	 0.08093594169543536


In [34]:
# add column to dataframe:
df["predicted_score_mode"] = score_mode_predicted

# cast column to int
df.predicted_score_mode = df.predicted_score_mode.astype(int)

df

,id,essentia_dissonance_mean,essentia_dissonance_stdev,essentia_dynamic_complexity,essentia_loudness,essentia_onset_rate,essentia_pitch_salience_mean,essentia_pitch_salience_stdev,essentia_spectral_centroid_mean,essentia_spectral_centroid_stdev,essentia_spectral_complexity_mean,essentia_spectral_complexity_stdev,essentia_spectral_rolloff_mean,essentia_spectral_rolloff_stdev,essentia_strong_peak_mean,essentia_strong_peak_stdev,librosa_bpm,librosa_chroma_mean_0,librosa_chroma_var_0,librosa_chroma_pct_10_0,librosa_chroma_pct_50_0,librosa_chroma_pct_90_0,librosa_chroma_mean_1,librosa_chroma_var_1,librosa_chroma_pct_10_1,librosa_chroma_pct_50_1,librosa_chroma_pct_90_1,librosa_chroma_mean_2,librosa_chroma_var_2,librosa_chroma_pct_10_2,librosa_chroma_pct_50_2,librosa_chroma_pct_90_2,librosa_chroma_mean_3,librosa_chroma_var_3,librosa_chroma_pct_10_3,librosa_chroma_pct_50_3,librosa_chroma_pct_90_3,librosa_chroma_mean_4,librosa_chroma_var_4,librosa_chroma_pct_10_4,...,librosa_chroma_mean_8,librosa_chroma_var_8,librosa_chroma_pct_10_8,librosa_chroma_pct_50_8,librosa_chroma_pct_90_8,librosa_chroma_mean_9,librosa_chroma_var_9,librosa_chroma_pct_10_9,librosa_chroma_pct_50_9,librosa_chroma_pct_90_9,librosa_chroma_mean_10,librosa_chroma_var_10,librosa_chroma_pct_10,librosa_chroma_pct_50_10,librosa_chroma_pct_90_10,librosa_chroma_mean_11,librosa_chroma_var_11,librosa_chroma_pct_10_11,librosa_chroma_pct_50_11,librosa_chroma_pct_90_11,librosa_spectral_bandwidth_mean,librosa_spectral_bandwidth_stdev,librosa_spectral_flatness_mean,librosa_spectral_flatness_stdev,midlevel_features_melody,midlevel_features_articulation,midlevel_features_rhythm_complexity,midlevel_features_rhythm_stability,midlevel_features_dissonance,midlevel_features_tonal_stability,midlevel_features_minorness,score_mode,score_key_strength,mean_A,mean_V,quadrant,predicted_mean_A,predicted_mean_V,predicted_quadrant,predicted_score_mode
0,GG-01-000,0.206445,0.131383,13.998791,58.329521,3.1,0.539736,0.193987,3367.790527,3684.124268,2.986095,2.530877,4423.862305,6692.707031,0.316206,0.329171,246.093750,0.423029,0.146997,0.008853,0.287094,1.000000,0.339058,0.089543,0.007493,0.289652,0.783410,0.372997,0.151724,0.008017,0.119889,1.000000,0.380705,0.097001,0.015079,0.304984,0.855147,0.471226,0.157847,0.020450,...,0.295291,0.087601,0.004258,0.190495,0.764857,0.300751,0.138661,0.001662,0.058367,0.975641,0.253962,0.094545,0.004239,0.074733,0.765817,0.301682,0.099494,0.006088,0.144401,0.815172,3455.577210,1781.705789,0.139911,0.248388,0.065397,-0.029380,-0.303987,-0.053306,-0.325228,0.347745,-0.119278,0.0,0.795727,50.689655,2.724138,1,45.216172,1.749838,4,0
1,GG-01-001,0.145753,0.057971,3.533264,75.166183,4.2,0.476832,0.123564,1395.528809,401.987610,4.181923,2.296265,691.307373,194.689529,0.438665,0.363517,246.093750,0.270370,0.112473,0.008509,0.087995,0.937823,0.191162,0.039303,0.008384,0.107667,0.472149,0.280426,0.145651,0.011143,0.075144,1.000000,0.281440,0.038196,0.023332,0.270883,0.559445,0.433210,0.177601,0.013791,...,0.141696,0.026381,0.005053,0.049705,0.425738,0.124960,0.059223,0.004241,0.040996,0.311990,0.128249,0.030747,0.005173,0.045485,0.455922,0.203408,0.069066,0.006298,0.093133,0.499201,2745.545697,721.292781,0.000275,0.000484,0.353040,-0.073957,-0.381182,0.120750,-0.479266,0.451572,-0.064252,0.0,0.844387,50.689655,2.724138,1,42.078254,0.360194,4,0
2,GG-01-002,0.149111,0.059011,3.337368,82.753929,4.3,0.457741,0.132359,1318.759644,289.306152,4.303592,2.127036,674.839294,210.393951,0.534696,0.410875,246.093750,0.193968,0.078629,0.003699,0.045964,0.661004,0.196718,0.045585,0.006544,0.134951,0.538369,0.394663,0.185849,0.008034,0.111176,1.000000,0.252466,0.056374,0.015938,0.169116,0.649735,0.160429,0.087263,0.009318,...,0.199484,0.028600,0.010491,0.160100,0.427959,0.275843,0.136039,0.009648,0.091636,1.000000,0.213787,0.054531,0.008481,0.097247,0.541696,0.172331,0.083168,0.005382,0.043987,0.575106,2593.570349,583.569947,0.000202,0.000314,0.201885,0.016944,-0.337422,0.103853,-0.388214,0

In [35]:
correct_score_mode = sum(df['predicted_score_mode'] == df['score_mode']) / len(df)
print("Percentage of Samples with Mode correctly predicted:", correct_score_mode)

Percentage of Samples with Mode correctly predicted: 0.9190640583045646


In [36]:
df.to_csv("dataset_with_score_mode_predicted.csv", index=False)

In [37]:
for i in df["predicted_score_mode"].values:
  print(i,end=", ")

0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0